# Get Tweet by brand

### 



In [5]:
# Tweepy - Python library for accessing the Twitter API.
import tweepy

# TextBlob - Python library for processing textual data
from textblob import TextBlob

# WordCloud - Python linrary for creating image wordclouds
from wordcloud import WordCloud

# NLTK - NLP library for sentiment analysis
import nltk

# Pandas - Data manipulation and analysis library
import pandas as pd

# NumPy - mathematical functions on multi-dimensional arrays and matrices
import numpy as np

# Regular Expression Python module
import re

# Time - Converting necessary time objects to desried formats
import time

# Matplotlib - plotting library to create graphs and charts
import matplotlib.pyplot as plt

# Settings for Matplotlib graphs and charts
from pylab import rcParams
rcParams['figure.figsize'] = 12, 8



In [98]:
# Set up consumer key and consumer secret
consumer_key = "5Ohy4gwfxvxaV7wPLj4LeRKSx"
consumer_secret = "lRewDNFpxLOKj4E1g63o5z2gEmticrFC4dM3tHatnh2xKl2iyC"

# Create authentication object and set access token & access token secret
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token("1326028749842780162-UhEVNo2wusr3W88eK2lfGsZGHV6fAM", 
    "N06snCbxez6OYTRZAQ33SGYybtPOoIVXoyTicWPqZOZjQ")

# Create the API object while passing in the auth information
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)
try:
    api.verify_credentials()
    print("Authentication OK")
except:
    print("Error during authentication")
    

Authentication OK


### Build function to get tweets for McPlant, McD, competitors, and vegan market

In [184]:
# Build Cursor function to get all the tweets mentioning kfc + vegan meat combination

def GetTweets(brands, number_of_tweets):
    meat_catalogue = ["vegan", "meat free", "vegetarian", "veg", "meatless", "plant based", "veggie"]
#     meat_catalogue = ["plant", "vegan", "impossible", "Beyond",
#                       "gardein", "meat-free", "vegetarian", "vegetarians", 
#                       "veg", "meatless", "faux-meat", "artificial", 
#                       "plant based meat",  "meat substitute",  "fake meat", 
#                       "faux meat", "meat free", "plant-based meat", "imitation meat", "veggie"]
    
    query = []
    for brand in brands:
        query.append(brand)
        for combo in meat_catalogue:
             query.append(brand+" "+combo)

                
    # Creation of query method using parameters with a for loop
    tweets_list = []
    for keyword in query:
        tweets = tweepy.Cursor(api.search,q=keyword,lang='en',tweet_mode='extended').items(number_of_tweets)
        # Pulling information from tweets iterable object
        # Add or remove tweet information you want in the below list comprehension
        for tweet in tweets:
            tweets_list.append([tweet.full_text, tweet.created_at, tweet.id_str,
                         tweet.user.screen_name, tweet.user.id_str, tweet.user.location, tweet.retweet_count,
                         tweet.favorite_count, tweet.in_reply_to_status_id_str, tweet.in_reply_to_user_id_str,
                         tweet.user.followers_count, tweet.user.friends_count, tweet.coordinates, tweet.place,
                         tweet.entities['hashtags'], keyword])

    # Creation of dataframe from tweets_list
    # Add or remove columns as you remove tweet information
    tweets_df = pd.DataFrame(tweets_list,columns=['Tweet Text', 'Tweet Datetime', 'Tweet Id', 
                                                  'Twitter @ Name', 'User Id', 'User Location',
                                                  'Retweets', 'Favorites', 'Replied Tweet Id', 
                                                  'Replied Tweet User Id Str', 'User Follower Counts', 
                                                  'User Following Counts', 'Tweet Coordinates', 
                                                  'Place Info','Hashtags','Keyword'])

    # Checks if there are coordinates attached to tweets, if so extracts them
    tweets_df['Tweet Coordinates'] = tweets_df.apply(extract_coordinates,axis=1)

    # Checks if there is place information available, if so extracts them
    tweets_df['Place Info'] = tweets_df.apply(extract_place,axis=1)
    
    return tweets_df

### Build functions to extract coordinates, place info, categorize country and clean emoji

In [185]:
# Function created to extract coordinates from tweet if it has coordinate info
# Tweets tend to have null so important to run check

def extract_coordinates(row):
    if row['Tweet Coordinates']:
        return row['Tweet Coordinates']['coordinates']
    else:
        return None

In [8]:
# Function created to extract place such as city, state or country from tweet if it has place info
# Tweets tend to have null so important to run check

def extract_place(row):
    if row['Place Info']:
        return row['Place Info'].full_name
    else:
        return None

In [57]:
# Convert emojis to text
import emoji
def cleanemoji(tweet):
    tweet = emoji.demojize(tweet)
    tweet = tweet.replace(":", " ") #to remove all : from :flushed_face:
    tweet = tweet.replace("_", " ") #to remove _ from thumbs_up
    tweet = " ".join(tweet.split())
    return tweet


### Build function to extract hashtags from the uncleaned hashtag formats

In [ ]:
# Function created to extract hashtag if it has any hashtags info
# Tweets tend to have null so important to run check

def get_hash_tag_list(hashtags):
    hash_tag_list=[]
    if hashtags != []:
        for i in hashtags:
            hash_tag_list.append(i['text'])
        return hash_tag_list
    else:
        return None
    

************************************************************************************************************************************************************

### Get all KFC tweets and relevant information and save it to csv

In [178]:
# Get all KFC tweets and save it to dataframe object
kfc_brands = ['kfc']
kfc_tweets_df = GetTweets(kfc_brands, 20000)

In [110]:
# Pass KFC Tweets for hashtag extraction

kfc_tweets_df['Hashtag List'] = kfc_tweets_df['Hashtags'].apply(get_hash_tag_list)

In [157]:
# Saving the file to csv

kfc_tweets_df.to_csv('kfc_tweets.csv')

### Get all Burger King tweets and relevant information and save it to csv

In [189]:
# Get all Burger King tweets and save it to dataframe object

burgerking_brands = ['burger king']
burgerking_tweets_df = GetTweets(burgerking_brands, 20000)

In [156]:
# Pass Burger King Tweets for hashtag extraction

burgerking_tweets_df['Hashtag List'] = burgerking_tweets_df['Hashtags'].apply(get_hash_tag_list)

In [158]:
# Saving the file to csv

burgerking_tweets_df.to_csv('burgerking_tweets.csv')

### Get all PizzaHut tweets and relevant information and save it to csv

In [190]:
# Get all Pizzahut tweets and save it to dataframe object

pizzahut_brands = ['pizzahut']
pizzahut_tweets_df = GetTweets(pizzahut_brands, 20000)

In [182]:
# Pass PizzaHut Tweets for hashtag extraction

pizzahut_tweets_df['Hashtag List'] = pizzahut_tweets_df['Hashtags'].apply(get_hash_tag_list)

In [183]:
# Saving the file to csv

pizzahut_tweets_df.to_csv('pizzahut_tweets.csv')

### Get all McDonald's tweets and relevant information and save it to csv

In [ ]:
# Get all McDonald's tweets and save it to dataframe object

mcd_brands = ["McDonald's"]
mcd_tweets_df = GetTweets(mcd_brands, 20000)

In [ ]:
# Pass McDonald's Tweets for hashtag extraction

mcd_tweets_df['Hashtag List'] = mcd_tweets_df['Hashtags'].apply(get_hash_tag_list)

In [ ]:
# Saving the file to csv

mcd_tweets_df.to_csv('mcd_tweets.csv')

### Get all McPlant tweets and relevant information and save it to csv

### Get all Vegan Market tweets and relevant information and save it to csv

### Data Preprocessing

We are going to create a cleantext function that will:
- remove mentions
- remove hashtags
- remove retweets
- remove urls

In [ ]:
# Data cleansing without stemming - For Word Cloud Visualization

def cleantext_cloud(text):
    import re
    import contractions
    text = text.lower().strip() # lowercase all letters
    text = re.sub(r'@[A-za-z0-9]+','',text) # remove @mentions
    text = re.sub(r'RT[\s]+','',text) #remove retweet
    text = re.sub(r'https?://\S+','',text) #remove hyperlink
    text = re.sub(r'#','',text) #remove #

    text = text.replace('\n',' ')

    text = contractions.fix(text)

    text = re.sub("[.,!?:;-=*\"'“()_]", "", text) # remove puntucation
    text = re.sub("[0123456789]", "", text) # remove number

    text = text.replace('  ','')

    return text.strip()

In [ ]:
# Data cleansing with stemming - For Tweets

def cleantext(text):

    text = text.lower().strip() # lowercase all letters
    text = re.sub(r'@[A-za-z0-9]+','',text) # remove @mentions
    text = re.sub(r'RT[\s]+','',text) #remove retweet
    text = re.sub(r'https?://\S+','',text) #remove hyperlink
    text = re.sub(r'#','',text) #remove #

    text = text.replace('\n',' ')

    text = contractions.fix(text)

    stemmer = SnowballStemmer(language='english') # Stemming for prediction
    text = stemmer.stem(text)
    text = re.sub("[.,!?:;-=*\"'“()_]", "",text) # remove puntucation
    text = re.sub("[0123456789]", "", text) # remove number

    text = text.replace('  ','')

    return text.strip()

##### For KFC

In [ ]:
# 1. Apply cleantext across all Tweets (KFC)

kfc_tweets_df['Clean_text'] = kfc_tweets_df['Tweet Text'].apply(cleantext)

In [ ]:
# 2. Demojize the tweets along Clean_text column

kfc_tweets_df['Clean_text'] = kfc_tweets_df['Clean_text'].apply(cleanemoji)

In [ ]:
# 3. Drop NA rows with no Tweet ID

kfc_tweets_df = kfc_tweets_df.dropna(thresh=3)

In [ ]:
# 4. Create another KFC pure brand tweets with filtering Keyword = kfc only for brand level analysis

kfc_brand_df = kfc_df.query("Keyword == 'kfc'")

##### For Burger King

In [ ]:
# 1. Apply cleantext across all Tweets (Burger King)

burgerking_tweets_df['Clean_text'] = burgerking_tweets_df['Tweet Text'].apply(cleantext)

In [ ]:
# 2. Demojize the tweets along Clean_text column

burgerking_tweets_df['Clean_text'] = burgerking_tweets_df['Clean_text'].apply(cleanemoji)

In [ ]:
# 3. Drop NA rows with no Tweet ID

burgerking_tweets_df = burgerking_tweets_df.dropna(thresh=3)

In [ ]:
# 4. Create another Burger King pure brand tweets with filtering Keyword = burgerking only for brand level analysis

burgerking_brand_df = burgerking_tweets_df.query("Keyword == 'burger king'")

##### For PizzaHut

In [ ]:
# 1. Apply cleantext across all Tweets (PizzaHut)

pizzahut_tweets_df['Clean_text'] = pizzahut_tweets_df['Tweet Text'].apply(cleantext)

In [ ]:
# 2. Demojize the tweets along Clean_text column

pizzahut_tweets_df['Clean_text'] = pizzahut_tweets_df['Clean_text'].apply(cleanemoji)

In [ ]:
# 3. Drop NA rows with no Tweet ID

pizzahut_tweets_df = pizzahut_tweets_df.dropna(thresh=3)

In [ ]:
# 4. Create another PizzaHut pure brand tweets with filtering Keyword = pizzahut only for brand level analysis

pizzahut_brand_df = pizzahut_tweets_df.query("Keyword == 'pizzahut'")

##### For Mcdonald's

In [ ]:
# 1. Apply cleantext across all Tweets (McDonald's)

mcd_tweets_df['Clean_text'] = mcd_tweets_df['Tweet Text'].apply(cleantext)

In [ ]:
# 2. Demojize the tweets along Clean_text column

mcd_tweets_df['Clean_text'] = mcd_tweets_df['Clean_text'].apply(cleanemoji)

In [ ]:
# 3. Drop NA rows with no Tweet ID

mcd_tweets_df = mcd_tweets_df.dropna(thresh=3)

In [ ]:
# 4. Create another McDonald's pure brand tweets with filtering Keyword = McDonald's only for brand level analysis

mcd_brand_df = mcd_df.query('Keyword == "McDonald\'s"')

##### For McPlant

In [ ]:
# 1. Apply cleantext across all Tweets (McPlant)

mcplant_tweets_df['Clean_text'] = mcplant_tweets_df['Tweet Text'].apply(cleantext)

In [ ]:
# 2. Demojize the tweets along Clean_text column

mcplant_tweets_df['Clean_text'] = mcplant_tweets_df['Clean_text'].apply(cleanemoji)

In [ ]:
# 3. Drop NA rows with no Tweet ID

mcplant_tweets_df = mcplant_tweets_df.dropna(thresh=3)

In [ ]:
# 4. Create another McPlant pure brand tweets with filtering Keyword = McPlant only for McPlant level analysis

mcplant_df = mcplant_tweets_df.query('Keyword == "McPlant"')

##### For Vegan Market

In [ ]:
# 1. Apply cleantext across all Tweets (Vegan Market)

vegan_tweets_df['Clean_text'] = vegan_tweets_df['Tweet Text'].apply(cleantext)

In [ ]:
# 2. Demojize the tweets along Clean_text column

vegan_tweets_df['Clean_text'] = vegan_tweets_df['Clean_text'].apply(cleanemoji)

In [ ]:
# 3. Drop NA rows with no Tweet ID

vegan_tweets_df = vegan_tweets_df.dropna(thresh=3)

************************************************************************************************************************************************

### Combine all pure brand dataframes for sentiment analysis through LSTM model later

In [ ]:
# Concatenate all brand dfs for getting sentiment scores through the LSTM model

combined_brand_df = pd.concat([kfc_brand_df, burgerking_brand_df, pizzahut_brand_df, mcd_brand_df],axis=0)

********************************************************************************************************************************************

### LSTM Model (Transfer Learning)

In [ ]:
# Load LSTM model for prediction

import tensorflow as tf
import tensorflow.keras
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer

model = tf.keras.models.load_model("twitter_sentiment")

In [ ]:
# Open tokenizer

import pickle

with open('tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

****************************************************************************************************************************

### Label Classification under each level - McPlant, McDonald's, Brand/Competitors, Vegan Market

#### Build the sentiment classifcation function first

In [ ]:
# Classify sentiment class function

def sentiment_class(score):
    if score >= 0.5:
        return "Positive"
    else:
        return "Negative"

#### 1. For McPlant Level

In [ ]:
# Pass cleaned tweets in McPlant dataframe to the LSTM model 

from tensorflow.keras.preprocessing.sequence import pad_sequences

testArr = mcplant_df.Clean_text.to_numpy()
input_tok = pad_sequences(tokenizer.texts_to_sequences(testArr),maxlen = 200)
scores = model.predict(input_tok, verbose=1, batch_size=10000)

In [ ]:
# Generate sentiment scores for each tweet

score_df = pd.DataFrame(scores, columns=['Score'])

In [ ]:
# Update McPlant dataframe for McPlant analysis

mcplant_df = mcplant_df.reset_index(drop=True)
score_df = score_df.reset_index(drop=True)
mcplant_final_df = pd.concat([mcplant_df, score_df], axis=1)

#### 2. For Brand/Competitor Level

In [ ]:
# Pass cleaned tweets in combined brand dataframe to the LSTM model 

from tensorflow.keras.preprocessing.sequence import pad_sequences

testArr = combined_brand_df.Clean_text.to_numpy()
input_tok = pad_sequences(tokenizer.texts_to_sequences(testArr),maxlen = 200)
scores = model.predict(input_tok, verbose=1, batch_size=10000)

In [ ]:
# Generate sentiment scores for each tweet

score_df = pd.DataFrame(scores, columns=['Score'])

In [ ]:
# Create final dataframe for overall brand analysis

mcplant_df = mcplant_df.reset_index(drop=True)
score_df = score_df.reset_index(drop=True)
brands_final_df = pd.concat([combined_brand_df, score_df], axis=1)

brands_final_df.to_csv('brands_final_df.csv')